In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/random-user/sample_user.csv
/kaggle/input/random-user/__results__.html
/kaggle/input/random-user/__notebook__.ipynb
/kaggle/input/random-user/__output__.json
/kaggle/input/random-user/custom.css
/kaggle/input/splitdata/test.h5
/kaggle/input/splitdata/__results__.html
/kaggle/input/splitdata/train.h5
/kaggle/input/splitdata/__notebook__.ipynb
/kaggle/input/splitdata/__output__.json
/kaggle/input/splitdata/custom.css


In [2]:
# Thiết lập để hiển thị tối đa số cột (None có nghĩa là không giới hạn)
pd.set_option('display.max_columns', None)

# Thiết lập để hiển thị toàn bộ chiều rộng cột (tùy chọn)
pd.set_option('display.width', None)

# Đọc file

In [3]:
df = pd.read_hdf("/kaggle/input/splitdata/train.h5",key='df')
test = pd.read_hdf("/kaggle/input/splitdata/test.h5", key="df")

## Thay bằng giá trị đặc biệt

In [4]:
new_df=df.copy()

In [5]:
# Chuẩn hóa giá trị về string và xử lý điều kiện
new_df['course_prerequisites'] = df['course_prerequisites'].astype(str).apply(
    lambda x: "no prerequisite" if x.strip() == "" or x.strip().lower().startswith("no") else x
)
test['course_prerequisites'] = test['course_prerequisites'].astype(str).apply(
    lambda x: "no prerequisite" if x.strip() == "" or x.strip().lower().startswith("no") else x
)

In [6]:
# Danh sách các cột cần xử lý
list_columns = [
    'school_about',
    'school_motto',
    'teacher_about',
    'teacher_job_title',
    'teacher_org_name'
]

# Hàm xử lý danh sách, thay giá trị khuyết bằng "unknown"
def fill_missing_in_list(lst):
    if isinstance(lst, list):
        return ["unknown" if pd.isna(item) else item for item in lst]
    return lst  # Giữ nguyên nếu không phải list

# Áp dụng cho từng cột trong danh sách
for col in list_columns:
    new_df[col] = new_df[col].apply(fill_missing_in_list)
    test[col] = test[col].apply(fill_missing_in_list)

In [7]:
# Thay các phần tử khuyết trong list bằng "unknown"
def fill_missing_in_list(lst):
    if isinstance(lst, list):
        return ["unknown" if pd.isna(x) else x for x in lst]
    return lst

# Áp dụng cho cột 'teacher_name'
new_df['teacher_name'] = new_df['teacher_name'].apply(fill_missing_in_list)
test['teacher_name'] = test['teacher_name'].apply(fill_missing_in_list)

# Xóa các cột có tỉ lệ null > 70% sau đó xóa các dòng chứa giá trị null

## Xóa cột

In [8]:
def drop_cols(df, nguong=0.5):
    """
    Xử lý explode các cột kiểu list, tính tỷ lệ null và xóa cột nếu tỷ lệ null vượt ngưỡng.
    
    Args:
        df (pd.DataFrame): DataFrame đầu vào.
        nguong (float): Ngưỡng tỷ lệ null cho phép (từ 0 đến 1).
    
    Returns:
        pd.DataFrame: DataFrame sau xử lý.
    """
    df_copy = df.copy()

    # Tính tỷ lệ null
    ty_le_null = df_copy.isnull().mean()
    print("Tỷ lệ null trên từng cột:")
    print(ty_le_null)

    # Xác định và in ra các cột bị xóa
    cot_bi_xoa = ty_le_null[ty_le_null > nguong].index.tolist()
    print(f"Các cột bị xóa (tỷ lệ null > {nguong}): {cot_bi_xoa}")

    # Trả về DataFrame đã xóa cột
    cot_giu_lai = ty_le_null[ty_le_null <= nguong].index
    return df_copy[cot_giu_lai]

In [9]:
new_df = drop_cols(new_df)
test = drop_cols(test)

Tỷ lệ null trên từng cột:
user_id                    0.000000
course_id                  0.000000
enroll_time                0.000000
course_name                0.000000
course_prerequisites       0.000000
course_about               0.000000
course_field               0.731185
school_id                  0.003490
school_name                0.003490
school_about               0.003490
school_motto               0.003490
teacher_id                 0.001178
teacher_name               0.001178
teacher_about              0.001178
teacher_job_title          0.001178
teacher_org_name           0.001178
video_id                   0.376540
video_ccid                 0.376540
video_duration             0.376540
user_gender                0.000000
watch_id                   0.999487
total_watch_time           0.999487
actual_watch_time          0.999487
exercise_id                0.019345
exercise_num_problem       0.019345
doing_exercise_id          0.997219
doing_num_correct          0.997219
co

## Xóa dòng

In [10]:
course_cols = new_df.columns[new_df.columns.str.contains('^(?:course|school|teacher|video|exercise|concept)', regex=True)].tolist()
course_df=new_df[course_cols].groupby('course_id').first()
course_df['count_course'] = new_df.groupby('course_id').size()
course_df=course_df.reset_index()
display(course_df.shape,course_df.head())

(2766, 20)

,course_id,course_name,course_prerequisites,course_about,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration,exercise_id,exercise_num_problem,course_total_comments,count_course
0,C_1017355,introduction to traditional chinese medicine e...,the main subjects are college students who hav...,traditional chinese medicine culture - the tre...,[S_33],[guangzhou university traditional chinese medi...,[guangzhou university chinese medicine first b...,[unknown],"[T_7970, T_7971, T_5826, T_7978, T_7979]","[su hong, liu shuaishuai, zhang xiaohu, liu, m...",[master english linguistics currently dean sch...,"[associate professor, lecturer, associate prof...",[guangzhou university traditional chinese medi...,None,None,None,"[Ex_1617861, Ex_1736572, Ex_1617873, Ex_161788...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",39624,1462
1,C_1017419,smart government talk,no prerequisite,this course combines specific cases of traffic...,[S_52],[central south university nationality],[southcentral minzu university referred scmzu ...,[unknown],"[T_7849, T_16199, T_16200, T_16201, T_16202, T...","[hu xinli, wu kaisong, peng qingjun, fang fuji...",[doctor management associate professor directo...,"[associate professor, professor, professor, as...","[central south university nationality, central...",None,None,None,"[Ex_1618263, Ex_1618257, Ex_1618248, Ex_161826...","[15, 15, 20, 15, 15, 15, 15, 16, 15]",609,116
2,C_1025064,principles of ideological and political education,no prerequisite,"hello, dear classmate! do you want to know how...",[S_13],[central south university],[central south university located changsha cit...,[integration knowledge action practice world],"[T_3915, T_3911, T_5426, T_5427, T_5429]","[liu wei, liu yongwei, lin jie, guo pengfei, c...",[liu wei male born hengnan county hunan provin...,"[associate professor, lecturer, lecturer, lect...","[central south university, central south unive...","[V_1622637, V_1622640, V_1622622, V_1622571, V...","[0B63D5E3D4D034E89C33DC5901307461, 0B6DEBC0F80...","[466.575, 402.325, 476.65, 1025.9, 755.075, 42...","[Ex_1622651, Ex_1622617, Ex_1622586, Ex_162263...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...",696,1252
3,C_1025076,advanced roller skating tutorial,no prerequisite,the true meaning of experiencing roller skatin...,[S_72],[northeastern university],[northeastern university northeastern universi...,[unknown],[T_5394],[li zhongshan],[fulltime teacher roller skating program north...,[lecturer],[northeastern university],None,None,None,"[Ex_1622870, Ex_1622876, Ex_1622875, Ex_1622889]","[1, 1, 15, 1]",2,150
4,C_1025079,scientific fitness method-silk-winding fitness...,no prerequisite,silk wrapping skills are the chinese solution ...,[S_97],[beijing sport university],[beijing sport university referred beijing spo...,[unknown],[T_7939],[wu dong],[professor phd education phd supervisor famous...,[professor],[beijing sport university],"[V_1623000, V_1622990, V_1622999, V_1623003, V...","[0805D199A89E1D799C33DC5901307461, 1026CDC4B72...","[671.97, 925.17, 748.11, 644.25, 543.93, 546.3...","[Ex_1623011, Ex_1622993, Ex_1623006, Ex_162302...","[5, 5, 5, 5, 5]",125,631


In [11]:
import pandas as pd

def is_effectively_null(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return True
    if isinstance(x, list):
        return any(pd.isna(i) for i in x)
    return False

def compute_null_stats(df):
    cols = df.columns
    isnull_rate = df.isnull().mean()

    effectively_null_rate = {}
    for col in cols:
        # Nếu kiểu dữ liệu của cột là list
        non_null_values = df[col].dropna()
        if not non_null_values.empty and isinstance(non_null_values.iloc[0], list):
            effectively_null_rate[col] = df[col].dropna().apply(is_effectively_null).mean()
        else:
            effectively_null_rate[col] = 0.0

    null_df = pd.DataFrame({
        'column': cols,
        'isnull_rate': [round(isnull_rate[col], 6) for col in cols],
        'effectively_null_rate': [round(effectively_null_rate[col], 6) for col in cols]
    })
    return null_df

null_stats = compute_null_stats(course_df)
display(null_stats)

,column,isnull_rate,effectively_null_rate
0,course_id,0.000000,0.0
1,course_name,0.000000,0.0
2,course_prerequisites,0.000000,0.0
3,course_about,0.000000,0.0
4,school_id,0.014461,0.0
5,school_name,0.014461,0.0
6,school_about,0.014461,0.0
7,school_motto,0.014461,0.0
8,teacher_id,0.005785,0.0
9,teacher_name,0.005785,0.0


In [12]:
new_df.shape

(6917397, 24)

In [13]:
new_df = new_df[new_df.isnull().sum(axis=1) <= int(0.25*new_df.shape[1]) ]

In [14]:
null_stats = compute_null_stats(course_df)
display(null_stats)

,column,isnull_rate,effectively_null_rate
0,course_id,0.000000,0.0
1,course_name,0.000000,0.0
2,course_prerequisites,0.000000,0.0
3,course_about,0.000000,0.0
4,school_id,0.014461,0.0
5,school_name,0.014461,0.0
6,school_about,0.014461,0.0
7,school_motto,0.014461,0.0
8,teacher_id,0.005785,0.0
9,teacher_name,0.005785,0.0


In [15]:
new_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 6900189 entries, 0 to 6917396
Data columns (total 24 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   user_id                  6900189 non-null  object
 1   course_id                6900189 non-null  object
 2   enroll_time              6900189 non-null  int64 
 3   course_name              6900189 non-null  object
 4   course_prerequisites     6900189 non-null  object
 5   course_about             6900189 non-null  object
 6   school_id                6886457 non-null  object
 7   school_name              6886457 non-null  object
 8   school_about             6886457 non-null  object
 9   school_motto             6886457 non-null  object
 10  teacher_id               6899965 non-null  object
 11  teacher_name             6899965 non-null  object
 12  teacher_about            6899965 non-null  object
 13  teacher_job_title        6899965 non-null  object
 14  teacher

In [16]:
from tqdm import tqdm

# Khởi động tqdm cho pandas
tqdm.pandas()

# Bước 1: Lấy danh sách theo 4 cột
cols = ['course_id', 'video_id', 'video_ccid', 'video_duration']
df_cleaned = new_df[cols].copy()

# Bước 2: Loại bỏ dòng trùng course_id (giữ dòng đầu tiên)
df_cleaned = df_cleaned.drop_duplicates(subset=['course_id'])

# Bước 3: Đảm bảo các cột là list (có tiến trình)
for col in ['video_id', 'video_ccid', 'video_duration']:
    df_cleaned[col] = df_cleaned[col].progress_apply(lambda x: x if isinstance(x, list) else [])

# Bước 4: Bung từng danh sách ra thành dòng riêng
df_exploded = df_cleaned.explode(['video_id', 'video_ccid', 'video_duration'])

# Bước 5: Loại bỏ các dòng có giá trị thiếu (có tiến trình)
df_exploded = df_exploded.progress_apply(
    lambda row: row if pd.notna(row['video_id']) and pd.notna(row['video_ccid']) and pd.notna(row['video_duration']) else None,
    axis=1
).dropna()

# Bước 6: Gom lại theo course_id và gom các giá trị về list (có tiến trình)
df_grouped = df_exploded.groupby('course_id').progress_apply(
    lambda g: pd.Series({
        'video_id': list(g['video_id']),
        'video_ccid': list(g['video_ccid']),
        'video_duration': list(g['video_duration'])
    })
).reset_index()

# Bước 7: Gộp lại vào new_df
new_df = new_df.drop(columns=['video_id', 'video_ccid', 'video_duration'], errors='ignore')
new_df = new_df.merge(df_grouped, on='course_id', how='left')

100%|██████████| 1292/1292 [00:00<00:00, 3223.27it/s]


In [17]:
# # Tập hợp tất cả các phần tử từ các danh sách trong cột
# all_items = new_df['video_duration'].dropna().explode()

# # Lấy các giá trị duy nhất và in ra
# unique_items = all_items.dropna().unique()

# # In từng giá trị
# for item in unique_items:
#     print(item)

In [18]:
# # Bung toàn bộ phần tử trong các list thành từng dòng
# exploded_items = new_df['teacher_name'].explode()

# # Tổng số phần tử sau khi explode
# total_count = len(exploded_items)

# # Số phần tử bị thiếu
# null_count = exploded_items.isna().sum()

# # Tính tỷ lệ phần trăm
# null_percent = (null_count / total_count) * 100 if total_count > 0 else 0

# # In kết quả
# print(f"Tổng số phần tử: {total_count}")
# print(f"Số phần tử thiếu (NaN): {null_count}")
# print(f"Tỷ lệ thiếu: {null_percent:.2f}%")

## Thay bằng giá trị đặc biệt

In [19]:
# Chuẩn hóa giá trị về string và xử lý điều kiện
new_df['course_prerequisites'] = new_df['course_prerequisites'].astype(str).apply(
    lambda x: "no prerequisite" if x.strip() == "" or x.strip().lower().startswith("no") else x
)

In [20]:
# Danh sách các cột cần xử lý
list_columns = [
    'school_about',
    'school_motto',
    'teacher_about',
    'teacher_job_title',
    'teacher_org_name'
]

# Hàm xử lý danh sách, thay giá trị khuyết bằng "unknown"
def fill_missing_in_list(lst):
    if isinstance(lst, list):
        return ["unknown" if pd.isna(item) else item for item in lst]
    return lst  # Giữ nguyên nếu không phải list

# Áp dụng cho từng cột trong danh sách
for col in list_columns:
    new_df[col] = new_df[col].apply(fill_missing_in_list)

In [21]:
# Thay các phần tử khuyết trong list bằng "unknown"
def fill_missing_in_list(lst):
    if isinstance(lst, list):
        return ["unknown" if pd.isna(x) else x for x in lst]
    return lst

# Áp dụng cho cột 'teacher_name'
new_df['teacher_name'] = new_df['teacher_name'].apply(fill_missing_in_list)

In [22]:
new_df.head(10)

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,user_gender,exercise_id,exercise_num_problem,course_total_comments,user_course_num_comment,user_course_num_replies,video_id,video_ccid,video_duration
0,U_1000129,C_735210,1599328511,university physics-electromagnetism (teaching ...,calculus newtonian mechanics,"physics, explore the laws of nature, and pursu...",[S_148],[yunnan university],[yunnan university abbreviated yunnan universi...,[unknown],[T_4242],[hu jianqiao],[hu jianqiao associate professor bachelor degr...,[associate professor],[yunnan university],0,"[Ex_635488, Ex_635480, Ex_635544, Ex_635491, E...","[5, 4, 2, 4, 7, 5, 5, 1, 6, 2, 5, 2, 1, 7, 3, ...",461,0,0,"[V_635512, V_635472, V_635516, V_635495, V_635...","[196B00DFA5515EB09C33DC5901307461, 1974570A080...","[889.363, 578.897, 404.653, 686.529, 429.572, ..."
1,U_1000129,C_799796,1599328642,"university physics 1 (mechanics, thermal)",no prerequisite,university physics is a basic physics course f...,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],"[T_164, T_165, T_166, T_167, T_168, T_169]","[yu, chen xinyi, zhang liuwan, wang shanying, ...",[yu professor doctoral supervisor engaged basi...,"[professor, professor, professor, associate pr...","[tsinghua university, tsinghua university, tsi...",0,"[Ex_822309, Ex_822340, Ex_822311, Ex_822327, E...","[4, 23, 7, 4, 5, 6, 10, 1, 8, 3, 5, 4, 4, 4, 1...",448,0,0,"[V_822227, V_822159, V_822282, V_822255, V_822...","[009035DB2BED87229C33DC5901307461, 015AC6C3159...","[237.67, 582.588, 405.612, 157.22, 392.304, 45..."
2,U_1000129,C_1925903,1599403037,mathematical analysis ii,mathematical analysis i,the mooc course in mathematical analysis ii ma...,[S_76],[beijing jiaotong university],[beijing jiaotong university national key univ...,[unknown],[T_12975],[song shichang],[since taught department mathematics school sc...,[associate professor],[beijing jiaotong university],0,"[Ex_6731684, Ex_6729452, Ex_6729437, Ex_672939...","[4, 1, 10, 5, 3, 7, 5, 1, 1, 7, 7, 7, 1, 6, 2,...",1,0,0,NaN,NaN,NaN
3,U_1000129,C_1992970,1599403537,mathematical analysis iii,no prerequisite,the mooc course of mathematical analysis iii c...,[S_76],[beijing jiaotong university],[beijing jiaotong university national key univ...,[unknown],[T_12975],[song shichang],[since taught department mathematics school sc...,[associate professor],[beijing jiaotong university],0,"[Ex_6882416, Ex_6882349, Ex_6882375, Ex_688239...","[8, 4, 6, 4, 6, 5, 6, 5, 5, 5, 6, 6, 6, 6, 6, ...",4,0,0,NaN,NaN,NaN
4,U_1000129,C_680884,1602844869,complex function,no prerequisite,let students master the mathematical theoretic...,[S_31],[xian jiaotong university],[xian jiaotong university referred xian jiaoto...,[diligent study righteous inspiring perseveran...,"[T_1523, T_1524, T_1525]","[wu huizhuo, zhang hongying, yi yuan]",[present professor doctoral associate professo...,"[associate professor, professor, professor]","[xian jiaotong university, xian jiaotong unive...",0,"[Ex_136594, Ex_136559, Ex_136574, Ex_136545, E...","[13, 15, 14, 15, 14, 16]",42,0,0,NaN,NaN,NaN
5,U_1000342,C_883345,1580818833,data structure (part 1),no prerequisite,breeze plan 2021 [first term recruitment] is i...,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_805],[deng junhui],[deng junhui professor department computer sci...,[professor],[tsinghua university],0,"[Ex_1007852, Ex_1008098, Ex_1008198, Ex_100769...","[11, 2, 3, 1, 4, 2, 5, 1, 10, 2, 3, 10, 10, 3,...",1322,0,0,"[V_1007803, V_1008155, V_1008065, V_1008384, V...","[00CE59849A6B87B29C33DC5901307461, 00CE59849A6...","[173.09, 173.09, 312.0, 312.0, 333.86, 333.86,..."
6,U_1000342,C_696968,1580818924,life english reading and writing,no prerequisite,"use reading to promote 

In [23]:
list1=new_df.user_id.drop_duplicates().to_list()
uu=pd.read_csv('/kaggle/input/random-user/sample_user.csv')
list2 = uu.user_id.to_list()
common = set(list1) & set(list2)
len(common)

24088

In [24]:
result = list(set(list2) - set(list1))
print(result) 

['U_7649089', 'U_1061752', 'U_29661', 'U_10626640', 'U_8435420', 'U_28669444', 'U_15999945', 'U_6522021', 'U_3264137', 'U_18405149', 'U_31767195', 'U_4369389', 'U_22149204', 'U_28936600', 'U_15157859', 'U_8307857', 'U_26383877', 'U_30583990', 'U_33497409', 'U_29061902', 'U_11530063', 'U_26340083', 'U_15365952', 'U_30987835', 'U_4487832', 'U_99772', 'U_28499331', 'U_17124396', 'U_30921564', 'U_10731184', 'U_30824466', 'U_15075390', 'U_30303669', 'U_15430355', 'U_13063097', 'U_29927552', 'U_31018003', 'U_15154223', 'U_30078892', 'U_12583973', 'U_15065128', 'U_29586732', 'U_3064280', 'U_31161864', 'U_14142297', 'U_15014624', 'U_687524', 'U_12664379', 'U_31462275', 'U_29777810', 'U_34710203', 'U_10236742', 'U_31098391', 'U_31415856', 'U_31476263', 'U_34170547', 'U_32852277', 'U_30542684', 'U_14170712', 'U_2461111', 'U_10954623', 'U_28770732', 'U_15268534', 'U_17894078', 'U_29072157', 'U_32068749', 'U_10950817', 'U_7014483', 'U_8171270', 'U_33459533', 'U_29405145', 'U_16087611', 'U_11700734

In [25]:
len(list2)

24581

In [26]:
list3= test.user_id.drop_duplicates().to_list()

common = set(list3) & set(list2)
len(common)

24090

In [27]:
test = test[~test['user_id'].isin(result)]

In [28]:
df_merged = pd.concat([new_df, test], ignore_index=True)
df_merged.to_hdf("data_ver2.h5",key="df")

/tmp/ipykernel_13/2625990580.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['user_id', 'course_id', 'course_name', 'course_prerequisites',
       'course_about', 'school_id', 'school_name', 'school_about',
       'school_motto', 'teacher_id', 'teacher_name', 'teacher_about',
       'teacher_job_title', 'teacher_org_name', 'exercise_id',
       'exercise_num_problem', 'video_id', 'video_ccid', 'video_duration'],
      dtype='object')]

  df_merged.to_hdf("data_ver2.h5",key="df")


In [29]:
# os.makedirs("data_ver2", exist_ok=True)
# %cd data_ver2
# new_df.to_hdf("train.h5",key="df")
# test.to_hdf("test.h5",key="df")